# Studying Confidence Intervals in Binary Classification

The following study was done using Wisconsin Breast Cancer Data

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from fastai.tabular import *

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/breast-cancer-wisconsin-data/data.csv


Let's read the data.

I'm not sure what 'Unnamed: 32' means and it appears to be all NaNs so for the purpuse of this study I removed it all together since I'm not really interesting in building the most accurate model.

In [2]:
df = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')
df = df.drop('Unnamed: 32', axis=1)
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
x1 = np.random.rand(1, len(df))[0]
x2 = np.random.rand(1, len(df))[0]
y_aux = np.random.rand(1, len(df))[0]
y = y_aux > 0.5
rand_df = pd.DataFrame({'x1': x1, 'x2': x2, 'y': y})
rand_df[:5]

,x1,x2,y
0,0.076099,0.037743,True
1,0.674779,0.397229,True
2,0.332316,0.624636,True
3,0.009809,0.728101,False
4,0.519617,0.443257,False


### Split between test and train sets

In [4]:
y = df['diagnosis']
X = df.drop('diagnosis', axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

df_train = X_train.copy()
df_valid = X_valid.copy()
df_train.insert(loc=len(X_train.columns), column='diagnosis', value=y_train)
df_valid.insert(loc=len(X_valid.columns), column='diagnosis', value=y_valid)

In [5]:
rand_y = rand_df['y']
rand_X = rand_df.drop('y', axis=1)
rand_X_train, rand_X_valid, rand_y_train, rand_y_valid = train_test_split(rand_X, rand_y, test_size=0.2, random_state=42)

Let's take advantage of fastai's Categorify to turn our only categorical variable into one: 'diagnosis'

In [6]:
cont, cat = cont_cat_split(df)
tfm = Categorify(cat, cont)
tfm(df_train)
tfm(df_valid, test=True)
df_train['diagnosis'].cat.categories

Index(['B', 'M'], dtype='object')

'M', the label we want to classify, is coded as 1

Let's also fill in the missing using fastai's brilliant FillMissing class

In [7]:
tfm = FillMissing(cat, cont)
tfm(df_train)
tfm(df_valid, test=True)

In [8]:
X_train = df_train.drop('diagnosis', axis=1)
y_train = df_train.diagnosis

X_valid = df_valid.drop('diagnosis', axis=1)
y_valid = df_valid.diagnosis

### We train a simple model with no feature engineering

In [9]:
m = RandomForestClassifier(n_estimators=200, min_samples_leaf=1, max_features='sqrt', n_jobs=7, oob_score=True)

m.fit(X_train, y_train)

m.score(X_train,y_train), m.score(X_valid, y_valid), m.oob_score_

(1.0, 0.9649122807017544, 0.9626373626373627)

In [10]:
rand_m = RandomForestClassifier(n_estimators=200, min_samples_leaf=1, max_features='sqrt', n_jobs=7, oob_score=True)

rand_m.fit(rand_X_train, rand_y_train)

rand_m.score(rand_X_train,rand_y_train), rand_m.score(rand_X_valid, rand_y_valid), rand_m.oob_score_

(1.0, 0.5087719298245614, 0.5538461538461539)

After checking the validation set and the oob score making sure we are not overfitting, let's take a look at a more interesting metric for binary classification: F1 score

In [11]:
from sklearn.metrics import f1_score, recall_score, precision_score

preds = m.predict(X_valid)
f1_score(y_valid, preds, pos_label='M', average='binary')

0.9523809523809524

In [12]:
rand_preds = rand_m.predict(rand_X_valid)
f1_score(rand_y_valid, rand_preds, pos_label=1, average='binary')

0.5254237288135594

That is sorprisingly good considering no feature engineering was done, however let's see how confident we should be of each prediction

For each input we calculate de prediction of each individual tree and we stack them

In [13]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
len(preds[:, 0]), len(preds[0]), len(X_valid), len(y_valid)

(200, 114, 114, 114)

In [14]:
rand_preds = np.stack([t.predict(rand_X_valid) for t in rand_m.estimators_])
len(rand_preds[:, 0]), len(rand_preds[0]), len(rand_X_valid), len(rand_y_valid)

(200, 114, 114, 114)

## The Bootstrap Method

We set a number of iterations and define the sample size so we can resample each array of predictions

In [15]:
n_iterations = 300
n_size = int(len(preds) * 0.60)

In [16]:
from sklearn.utils import resample

Lowers = []
Uppers = []

for i in range(len(preds[0])):
    
    means = []
    
    for _ in range(n_iterations):
        rs = resample(preds[:, i], n_samples=n_size, replace=True)
        means.append(np.mean(rs))
    
    alpha = 0.99
    p = ((1.0 - alpha) / 2.0) * 100
    lower = max(0.0, np.percentile(means, p))
    Lowers.append(lower)
    
    p = (alpha + ((1.0 - alpha) / 2.0)) * 100
    upper = min(1.0, np.percentile(means, p))
    Uppers.append(upper)

In [17]:
rand_Lowers = []
rand_Uppers = []

for i in range(len(rand_preds[0])):
    
    means = []
    
    for _ in range(n_iterations):
        rs = resample(rand_preds[:, i], n_samples=n_size, replace=True)
        means.append(np.mean(rs))
    
    alpha = 0.99
    p = ((1.0 - alpha) / 2.0) * 100
    lower = max(0.0, np.percentile(means, p))
    rand_Lowers.append(lower)
    
    p = (alpha + ((1.0 - alpha) / 2.0)) * 100
    upper = min(1.0, np.percentile(means, p))
    rand_Uppers.append(upper)

### Now we have the upper and lower percentiles we can create a dataframe with statistical information about our predictions

In [18]:
X = pd.DataFrame({'actuals': y_valid.cat.codes,
                  'preds': np.mean(preds, axis=0),
                  'std': np.std(preds, axis=0),
                  'var': np.var(preds, axis=0),
                  'upper': Uppers - np.mean(preds, axis=0),
                  'lower': np.mean(preds, axis=0) - Lowers
                 })
X.reset_index(inplace=True)
X = X.drop('index', axis=1)
X[:10]

,actuals,preds,std,var,upper,lower
0,0,0.040,0.195959,0.038400,0.039208,0.031667
1,1,0.995,0.070534,0.004975,0.005000,0.020000
2,1,0.990,0.099499,0.009900,0.010000,0.031667
3,0,0.000,0.000000,0.000000,0.000000,0.000000
4,0,0.005,0.070534,0.004975,0.028333,0.005000
5,1,1.000,0.000000,0.000000,0.000000,0.000000
6,1,1.000,0.000000,0.000000,0.000000,0.000000
7,1,0.935,0.246526,0.060775,0.044208,0.064208
8,0,0.695,0.460407,0.211975,0.100875,0.111667
9,0,0.045,0.207304,0.042975,0.055000,0.036667


In [19]:
rand_X = pd.DataFrame({'actuals': rand_y_valid.astype(int),
                  'preds': np.mean(rand_preds, axis=0),
                  'std': np.std(rand_preds, axis=0),
                  'var': np.var(rand_preds, axis=0),
                  'upper': rand_Uppers - np.mean(rand_preds, axis=0),
                  'lower': np.mean(rand_preds, axis=0) - rand_Lowers
                 })
rand_X.reset_index(inplace=True)
rand_X = rand_X.drop('index', axis=1)
rand_X[:10]

,actuals,preds,std,var,upper,lower
0,0,0.475,0.499375,0.249375,0.100083,0.125083
1,0,0.790,0.407308,0.165900,0.085000,0.090000
2,1,0.300,0.458258,0.210000,0.112542,0.095875
3,0,0.370,0.482804,0.233100,0.117625,0.107542
4,1,0.730,0.443959,0.197100,0.095000,0.096667
5,1,0.435,0.495757,0.245775,0.127542,0.105875
6,1,0.675,0.468375,0.219375,0.091750,0.100000
7,1,0.550,0.497494,0.247500,0.108333,0.137542
8,1,0.805,0.396201,0.156975,0.090875,0.100875
9,0,0.410,0.491833,0.241900,0.098333,0.105875


Let's plot individual predictions using the average of the N trees and the percentiles as error bars

In [20]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    y=X['preds'][:50],
    error_y=dict(
            type='data',
            symmetric=False,
            array=X['upper'][:50],
            arrayminus=X['lower'][:50])))

fig.update_layout(shapes=[
    dict(type= 'line', yref='y', y0= 0.5, y1= 0.5, xref= 'x', x0= -1, x1= 50)])
fig.show()

In [21]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    y=rand_X['preds'][:50],
    error_y=dict(
            type='data',
            symmetric=False,
            array=rand_X['upper'][:50],
            arrayminus=rand_X['lower'][:50])))

fig.update_layout(shapes=[
    dict(type= 'line', yref='y', y0= 0.5, y1= 0.5, xref= 'x', x0= -1, x1= 50)])
fig.show()

## Uncertainty in our Models
### Just as a reminder, this was the F1 score for the hole validation dataset

In [22]:
f1_score(y_valid, m.predict(X_valid), pos_label='M', average='binary')

0.9523809523809524

If we now keep only those predictions whose average minus their lower percentile is greater than 0.7, and those whose average plus their upper percentile is smaller than 0.3 we can see the F1 score changes, going now to 98%

In [23]:
aux = X.copy()
aux = aux.loc[(aux['preds'] - aux['lower'] >= 0.7) | (aux['preds'] + aux['upper'] <= 0.3), :]
a = np.array(aux.preds > 0.5)

aux['prediction'] = 0

aux.loc[aux.preds > 0.5, 'prediction'] = 1


f1_score(aux.actuals, aux.prediction, pos_label=1, average='binary'), len(aux), len(X), len(aux)/len(X)

(1.0, 102, 114, 0.8947368421052632)

It's important to note that the 98% corresponds to 92% of the validation dataset, but the important part is that, **that 92% is not random**.

If we do the same but we adjust the lower and greater limits to 0.25 and 0.75 respectively we now get a perfect F1 score in our validation set

In [24]:
aux = X.copy()
aux = aux.loc[(aux['preds'] - aux['lower'] >= 0.75) | (aux['preds'] + aux['upper'] <= 0.25), :]
a = np.array(aux.preds > 0.5)

aux['prediction'] = 0

aux.loc[aux.preds > 0.5, 'prediction'] = 1


f1_score(aux.actuals, aux.prediction, pos_label=1, average='binary'), len(aux), len(X), len(aux)/len(X)

(1.0, 100, 114, 0.8771929824561403)

Instead we now see that this perfect F1 score represents only 87% of the validation set, but again, this 87% is not random. We know exactly which inputs represent this 87% since are the ones that our model trees classify with certain confidence. 
So you may ask, what about the remaining 13% of our data? Well, simply put, our model is not entirely sure.

The questions we should now be asking ourselves are: Is this validation set big enough? Is this validation set representative of what the model is going to see in production?

And finally, is this enough to determine if our model is making a mistake? Certainly not, but I think this analysis is a step in the right direction.

What do you think?

Any insights are welcome, you can reach me at tomi.ambro94@gmail.com